<a href="https://colab.research.google.com/github/mar1506/dotfiles/blob/master/Understand_embeddings_with_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>
<hr>

_Embeddings_ are representation of words thanks to vectors. These embeddings can be learnt within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which the ÌMDB dataset: it corresponds to sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In the previous exercise, we jointly learnt a representation for the words, and feed this representation to a RNN, as shown here : 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which can slow the convergence, and make it harder!

For that reason, we will separate the steps of learning the word representation and feeding it to a RNN. As shown here : 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with the word2vec.

The drawback is indeed that the learnt embedding is not _specifically_ designed for our task. However, learning it independently of the task at hand (sentiment analysis) has some advantages : 
- it is very fast to do in general (with word2vec)
- the representation learnt by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As on this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`.

In [2]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

Let's look at the embedded representation of some words.

You can use `word2vec.wv` as a dictionary.
For instance, `word2vec.wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation were not learn). 

In [3]:
word2vec.wv['brother']

array([ 0.07844488,  0.7644711 ,  0.21882778,  0.26382267,  0.01314416,
       -0.36933747, -0.08810634, -0.18189937,  0.19595887,  0.04287645,
        0.08303223,  0.05712103,  0.11947067, -0.2106274 , -0.30751988,
       -0.351758  , -0.27597633,  0.28098506, -0.17614807,  0.32202664,
       -0.00191581,  0.48451307, -0.09656357,  0.07547107, -0.10571094,
       -0.20504159,  0.9199318 ,  0.23830077,  0.21239984,  0.16932815,
       -0.5564228 ,  0.08761671,  0.03892357, -0.054103  ,  0.5103875 ,
        0.46677244, -0.16612162, -0.35028896,  0.43616167,  0.21878888,
       -0.20618138,  0.10868587,  0.28828385,  0.18492962,  0.3347238 ,
        0.41039327,  0.03952486, -0.7658746 ,  0.28887507, -0.8575522 ,
       -0.01432361,  0.51764023,  0.287696  ,  0.1224428 , -0.13979942,
        0.14586571,  0.14842182, -0.08920785,  0.21304904, -0.45436874,
       -0.3973817 , -0.57835084,  0.51074284,  0.34804463, -0.2725376 ,
        0.18568236, -0.19814743,  0.06859285, -0.19428805, -0.13

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [4]:
(word2vec.wv['bag']).shape


(100,)

How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the `word2vec.most_similar(...)` method that, given an input word, display the "closest" words in the embedding space. If the embedding is well done, then words that have close meanings will have close representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [5]:
word2vec.most_similar('brother', topn = 5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('boss', 0.9877901673316956),
 ("wife's", 0.9861291646957397),
 ('agent', 0.9858656525611877),
 ('secretary', 0.9855850338935852),
 ('affair', 0.9845020771026611)]

Similarly to `most_similar` used on words directly, we can use `similar_by_vector` on vectors to do the same thing :

In [6]:
word2vec.most_similar('brother')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('boss', 0.9877901673316956),
 ("wife's", 0.9861291646957397),
 ('agent', 0.9858656525611877),
 ('secretary', 0.9855850338935852),
 ('affair', 0.9845020771026611),
 ('baby', 0.9840225577354431),
 ('slowly', 0.9832121729850769),
 ("jane's", 0.9831498861312866),
 ('ex', 0.9830414652824402),
 ('body', 0.9827346205711365)]

# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [7]:
word2vec['good']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.28036812, -0.34944084, -1.0180323 ,  0.1000254 ,  0.7067004 ,
        0.3916027 ,  0.39437324,  0.26919165,  1.191167  ,  0.23404291,
        0.10433029, -0.52164143,  0.07894851,  0.1958071 , -0.4749633 ,
       -0.68388236, -0.36214727,  1.2183172 , -0.2909535 ,  0.07494736,
        1.027347  ,  0.94684625, -0.6375715 , -1.5083861 ,  0.19520283,
        0.64566016,  0.37898725, -0.31341583,  0.15045312,  0.43570265,
        0.2428458 ,  0.71079594,  0.05170619,  1.1074327 ,  0.45858735,
        0.55705863,  0.53201336, -0.50799155,  0.53521305, -0.4114679 ,
       -0.2652131 , -0.378881  , -0.57729524,  0.694329  ,  0.9288008 ,
        0.16157246, -0.5394942 , -0.55978453,  0.82730156,  0.328703  ,
        0.03652394, -0.73975646, -0.33993444, -0.6901197 ,  0.8938073 ,
        0.21640193, -0.5062941 , -1.1285013 ,  0.05574572,  0.36283973,
       -0.6822395 , -1.4646344 ,  0.7204129 ,  0.32502013,  1.3367996 ,
        0.1615151 ,  0.17371702,  1.6513917 , -0.27267498,  0.22

In [8]:
word2vec['bad']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.9758785 , -0.06054089, -0.8376446 , -0.1661509 ,  0.3839048 ,
        0.22498976,  0.7041599 ,  0.2341546 ,  1.2482345 ,  0.17135358,
       -0.20619692, -0.8202449 , -0.18652634,  0.85711986, -0.7692983 ,
       -0.79055214, -0.09010241,  1.1463437 , -0.60753727, -0.12711234,
        0.767033  ,  0.74863034, -0.7242605 , -2.0401    ,  0.17307363,
        0.7380483 ,  0.302094  , -0.2914538 , -0.23013993,  0.98624295,
       -0.5175133 ,  0.8201486 ,  0.00381414,  0.6945321 ,  0.75245047,
        0.43474776,  0.20485692, -0.52426404, -0.01444014,  0.04690975,
        0.16758966, -0.11424051, -0.560066  ,  0.36846387,  0.45380053,
        0.22311354, -0.6661848 , -0.479867  ,  0.746905  ,  0.20708627,
       -0.37188217, -0.76185226, -0.46062675, -0.9051041 ,  1.1570258 ,
        0.35051125, -0.49734926, -1.4377657 ,  0.51131254,  0.524433  ,
       -0.5205661 , -1.2062815 ,  0.6421526 ,  0.36308318,  1.6335195 ,
        0.67242414,  0.33857548,  2.1877418 , -0.36111674,  0.34

In [9]:
word2vec['good'] - word2vec['bad']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.6955104 , -0.28889996, -0.18038774,  0.26617628,  0.32279557,
        0.16661294, -0.30978668,  0.03503706, -0.05706751,  0.06268933,
        0.3105272 ,  0.29860348,  0.26547486, -0.66131276,  0.294335  ,
        0.10666978, -0.27204487,  0.07197344,  0.31658378,  0.2020597 ,
        0.260314  ,  0.1982159 ,  0.086689  ,  0.53171396,  0.02212919,
       -0.09238815,  0.07689324, -0.02196202,  0.38059306, -0.5505403 ,
        0.76035905, -0.10935265,  0.04789206,  0.41290063, -0.29386312,
        0.12231088,  0.32715642,  0.01627249,  0.5496532 , -0.45837766,
       -0.43280277, -0.2646405 , -0.01722926,  0.32586515,  0.4750003 ,
       -0.06154108,  0.12669057, -0.07991752,  0.08039653,  0.12161672,
        0.4084061 ,  0.0220958 ,  0.12069231,  0.21498442, -0.26321852,
       -0.13410932, -0.00894481,  0.30926442, -0.45556682, -0.16159329,
       -0.16167337, -0.25835288,  0.0782603 , -0.03806305, -0.2967199 ,
       -0.510909  , -0.16485846, -0.53635   ,  0.08844176, -0.12

Now, image for a second that, somehow, the following equality holds true - just for a second

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to 

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be foolish of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [10]:
res = word2vec.wv['good'] - word2vec.wv['bad'] + word2vec.wv['stupid']
res

array([ 0.36364865, -0.03988828, -0.303497  ,  0.2530726 ,  0.31436428,
        0.12568676, -0.05146831,  0.03145878,  0.49750447,  0.32976073,
        0.2653114 , -0.08813155,  0.28434494, -0.48726714, -0.2567947 ,
       -0.35392994, -0.31043866,  0.6738354 , -0.14091063,  0.08416631,
        0.42948145,  0.76987433, -0.26540524, -0.49734294,  0.01459266,
        0.20936596,  0.73968077, -0.1513848 ,  0.55912673, -0.03424275,
        0.27697885,  0.05877638,  0.16965869,  0.5056011 ,  0.20612586,
        0.50967914,  0.3703368 , -0.26267204,  0.74844664, -0.28041148,
       -0.41733098, -0.20401193, -0.17690891,  0.52693033,  0.7278209 ,
        0.13827384, -0.15301833, -0.6335592 ,  0.64276356, -0.24643114,
        0.24147645, -0.12870945,  0.00846568, -0.10625124,  0.09193674,
       -0.00472535, -0.14508727, -0.3800649 , -0.12519875, -0.12555963,
       -0.53051436, -0.9235814 ,  0.6066879 ,  0.26696125,  0.41087353,
       -0.22793606, -0.15508069,  0.36086798, -0.00548852, -0.17

We earlier said that, for any vector, it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `word2vec.wv.similar_by_vector` function) of `res`

In [11]:
word2vec.wv.similar_by_vector(res)

[('nice', 0.8190746307373047),
 ('good', 0.7813634276390076),
 ('potential', 0.7658889293670654),
 ('always', 0.7658437490463257),
 ('deed', 0.7636969089508057),
 ('considered', 0.7625093460083008),
 ('rotten', 0.7619683742523193),
 ('shallow', 0.7598617076873779),
 ('also', 0.75977623462677),
 ('casting', 0.7591575384140015)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [12]:
word2vec.wv['boy'] - word2vec.wv['girl'] 


array([ 0.02387837, -0.13334703,  0.0228833 , -0.2204645 , -0.2310467 ,
        0.24296752,  0.2251661 , -0.05537346, -0.03541873,  0.03381006,
       -0.02820839, -0.09484578, -0.10350215,  0.0430564 , -0.12043357,
       -0.11371687,  0.12966302, -0.43150723, -0.04432358,  0.07228571,
        0.11380366, -0.26751208, -0.01095181, -0.24488261,  0.14526623,
       -0.0753617 ,  0.06439596,  0.02249971,  0.13955536, -0.10460076,
        0.0738765 , -0.15326887,  0.04842924,  0.06963913,  0.16039947,
       -0.16562438,  0.12586877,  0.07717109, -0.25888538, -0.19389534,
        0.20971847, -0.05056566, -0.07214996,  0.00316104,  0.06076777,
       -0.20811966,  0.18991224, -0.03849351, -0.01343423, -0.0707857 ,
        0.03346885, -0.13701183, -0.2686939 , -0.27373856,  0.07821295,
       -0.0715012 , -0.18828082, -0.20102888,  0.18150723,  0.1256653 ,
        0.08012795,  0.14761823,  0.01198691, -0.10497043, -0.05643964,
       -0.01613759,  0.2278853 , -0.01986025,  0.3102513 ,  0.05

In [13]:
word2vec.wv['boy'] - word2vec.wv['girl'] 

array([ 0.02387837, -0.13334703,  0.0228833 , -0.2204645 , -0.2310467 ,
        0.24296752,  0.2251661 , -0.05537346, -0.03541873,  0.03381006,
       -0.02820839, -0.09484578, -0.10350215,  0.0430564 , -0.12043357,
       -0.11371687,  0.12966302, -0.43150723, -0.04432358,  0.07228571,
        0.11380366, -0.26751208, -0.01095181, -0.24488261,  0.14526623,
       -0.0753617 ,  0.06439596,  0.02249971,  0.13955536, -0.10460076,
        0.0738765 , -0.15326887,  0.04842924,  0.06963913,  0.16039947,
       -0.16562438,  0.12586877,  0.07717109, -0.25888538, -0.19389534,
        0.20971847, -0.05056566, -0.07214996,  0.00316104,  0.06076777,
       -0.20811966,  0.18991224, -0.03849351, -0.01343423, -0.0707857 ,
        0.03346885, -0.13701183, -0.2686939 , -0.27373856,  0.07821295,
       -0.0715012 , -0.18828082, -0.20102888,  0.18150723,  0.1256653 ,
        0.08012795,  0.14761823,  0.01198691, -0.10497043, -0.05643964,
       -0.01613759,  0.2278853 , -0.01986025,  0.3102513 ,  0.05

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)` , actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it chooses many splits in the different sentences, choose some words as inputs $X$  and a word as output $y$ which it tries to predict, in the embedding space.

And as any neural network, Word2Vec has some hyperparameters. Let's check some. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [14]:
word2vec_2 = Word2Vec(sentences=X_train, size=4*3)


In [15]:
word2vec_2.wv['brother']

array([ 0.64376754,  2.8919094 ,  0.61202884,  0.63728905, -0.32400402,
       -0.95701206, -0.67272455, -0.59447956,  0.10882372,  0.48578835,
        0.33955926,  0.22682025], dtype=float32)

❓ **Question** ❓ Use the `word2vec.wv.key_to_index` attribute to display the size of the learnt vocabulary. On the other hand, compare it to the number of different words in `X_train`.

In [16]:
len(word2vec_2.wv.index2word)

8006

In [17]:
words_total = []
for word in X_train:
  words_total += word

len(words_total)

582733

In [18]:
import numpy as np

In [19]:
len(np.unique(words_total))

30419

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been train on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec :  `min_count`. 

`min_count` is a integer that tells you how many occurences a given word should have to be learn in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vec that you have trained (you can choose any `vector_size` you want).

In [20]:
word2vec_3 = Word2Vec(sentences = X_train, size =12, min_count = 6)

In [21]:
word2vec_4 = Word2Vec(sentences= X_train, size = 12, min_count = 3)

Remember that we say that word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Learn a new `word2vec_5` model with a `window` different than previously (default is 5).

In [22]:
word2vec_5 = Word2Vec(sentences= X_train, size = 10, min_count = 4, window = 7 )

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the one that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here :

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [26]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
  res = []
  for word in sentence:
    try:
      embed = word2vec.wv[word]
      res.append(embed)
    except KeyError:
      pass
  return np.array(res)
    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentence (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [24]:
def embedding(word2vec, sentences):
  res = []
  for sentence in sentences:
    embedded_sentence = embed_sentence(word2vec, sentence)
    res.append(embedded_sentence)

  return res
    
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad them in order to have tensors that can be divided in batch sizes during the optimization. Store the padedd values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [25]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train)
X_test_pad = pad_sequences(X_test)

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)